# Open Power System Data: Renewable power plants

## Part 2: Validation and output

Part 1 of the script has downloaded and merged the original data from the sources.

Part 2 subsequently checks, validates the list of renewable power plants and creates CSV/XLSX/SQLite files. It also generates a daily timeseries of cumulated installed capacities by energy source.

*(Before running this script make sure you ran Part 1, so that the renewables.pickle files for each country exist in the same folder as the scripts)*


# Table of contents 

* [Part 1: Download and process](download_and_process.ipynb)
* [Script setup](#Script-setup)
* [4. Load data](#4.-Load-data)
* [5. Validation](#5.-Validation)
    * [5.1 Germany DE](#5.1-Germany DE)
* [6. Capacity time series](#6.-Capacity-time-series)
* [7. Plots](#7.-Plots)
    * [7.1 Cumulated capacity validation](#7.1-Cumulated-capacity-validation)
        * [7.1.1 Download and read data for validaion](#7.1.1-Download-and-read-data-for-validation)
        * [7.1.2 Prepare cumulated capacity for comparison](#7.1.2-Prepare-cumulated-capacity-for-comparison)
        * [7.1.3 Plot cumulated capacity](#7.1.3-Plot-cumulated-capacity)
    * [7.2 Capacity time series DE validation](#7.2-Capacity-time-series-DE-validation)
* [8. Harmonize format for output files](#8.-Harmonize-format-for-output-files)
    * [8.1 Columns](#8.1-Columns)
    * [8.2 Accurancy of numbers](#8.2-Accurancy-of-numbers)
    * [8.3 Sort](#8.3-Sort)
    * [8.4 Merge country data for SQL-output](#8.4-Merge-country-data-for-SQL-output)
* [9. Output](#9.-Output)
    * [9.1 Settings](#9.1-Settings)
    * [9.2 Write power plant lists](#9.2-Write-power-plant-list)
    * [9.3 Write time series](#9.3-Write-time-series)
    * [9.4 Write meta data](#9.4-Writing-meta-data)

# Script setup

In [2]:
# importing all necessary Python libraries for this Script

import json
import yaml
import os
import pandas as pd
import sqlite3
import logging
from bokeh.charts import Scatter, Line, Bar, show, output_file
from bokeh.io import output_notebook

output_notebook()

# Set up a log
logger = logging.getLogger('notebook')
logger.setLevel('INFO')
nb_root_logger = logging.getLogger()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s'\
                              '- %(message)s', datefmt='%d %b %Y %H:%M:%S')

# Create input and output folders if they don't exist
os.makedirs('input/original_data', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('output/datapackage_renewables', exist_ok=True)

Loading BokehJS ...

# 4. Load data

In [3]:
# Read data from script Part 1
DE_re = pd.read_pickle('DE_renewables.pickle')
DK_re = pd.read_pickle('DK_renewables.pickle')
FR_re = pd.read_pickle('FR_renewables.pickle')
PL_re = pd.read_pickle('PL_renewables.pickle')

# 5. Validation 

-- *Work in progress - all information on suspect data is welcome!* --

This section checks the data frame for a set of pre-defined criteria and adds markers to the entries in an additional column. The marked data will be included in the output files, but marked, so that they can be easiliy filtered out. For creating the validation plots and the timeseries, suspect data is skipped.

In [4]:
# Read csv of Marker Explanations
validation = pd.read_csv('input/validation_marker.csv',
                         sep=',', header=0)
validation

,Validation marker,Explanation
0,R_1,The data frame overlaps in date of the record ...
1,R_2,The start_up_date column has no entry (NA/Null)
2,R_3,"All power plant entries which do not have ""sta..."
3,R_4,start_up_date of PV system is older than 1975.
4,R_5,energy_source contains '#NV' value.
5,R_6,electrical_capacity is not specified.


## 5.1 Germany DE
**Main issue:** Due to the shortened zip-Code (5 to 3 digits) and eeg_id (33 to 15 characters) in the data from Netztransparenz (until end of 2015), we are not able to identify which of the power plants are also in the BNetzA-data. Thus the duplicate entries from August 2014 - end 2015 are not filtered, but all BNetzA-entries in this timespan marked.

**Add marker to data according to criteria (see above)**

In [5]:
# Create empty marker column
DE_re['comment'] = ""

# Validation criteria (R_1) for data source BNetzA
idx_date = DE_re[(DE_re['commissioning_date'] <= '2015-12-31') & (
                  DE_re['data_source'] == 'BNetzA')].index

# this can be replaced by renewables.loc[idx_date,'comment'] += "R_1, ".
# The same goes for the expressions below
DE_re.loc[idx_date, 'comment'] = DE_re.loc[idx_date, 'comment'] + "R_1, "

# Validation criteria (R_1) for source BNetzA_PV
idx_date_pv = DE_re[(DE_re['commissioning_date'] <= '2015-12-31')& (
              DE_re['data_source'] == 'BNetzA_PV')].index

DE_re.loc[idx_date_pv, 'comment'] += "R_1, "

# Validation criteria (R_2)
idx_date_null = DE_re[(DE_re['commissioning_date'].isnull())].index

DE_re.loc[idx_date_null, 'comment'] += "R_2, "

# Validation criteria (R_3)
idx_not_inst = DE_re[(DE_re['notification_reason'] != 'Inbetriebnahme') & (
                      DE_re['data_source'] == 'BNetzA')].index

DE_re.loc[idx_not_inst, 'comment'] += "R_3, "

# Validation criteria (R_4)
idx_pv_date = DE_re[(DE_re['commissioning_date'] < '1975-01-01') & (
                     DE_re['energy_source'] == 'solar')].index

DE_re.loc[idx_pv_date, 'comment'] += "R_4, "

# Validation criteria (R_5)
idx_nv = DE_re[DE_re['energy_source'] == '#NV'].index

DE_re.loc[idx_nv, 'comment'] += "R_5, "

# Validation criteria (R_6)
idx_capacity = DE_re[DE_re.electrical_capacity_kW <= 0.0].index

DE_re.loc[idx_capacity, 'comment'] += "R_6, "

**Create cleaned DataFrame**
All marked entries are deleted for the cleaned version of the DataFrame that is utilized for creating time series of installation and for the validation plots.

In [6]:
# Locate suspect entires
idx_suspect = DE_re[DE_re.comment.str.len() > 1].index

# create new data frame without suspect entries
DE_re_clean = DE_re.drop(idx_suspect)

**Overview suspect entries - cleaned data**

In [7]:
# Count marked entries
DE_re.groupby(['comment','data_source'])['comment'].count()

comment     data_source
            50Hertz        152187
            Amprion        471730
            BNetzA            868
            BNetzA_PV       17134
            TenneT         693798
            TransnetBW     322693
R_1,        BNetzA           3512
            BNetzA_PV       78521
R_1, R_3,   BNetzA           3580
R_2, R_3,   BNetzA           1276
R_3,        BNetzA              4
R_6,        50Hertz             3
            TransnetBW          5
Name: comment, dtype: int64

In [8]:
# Summarize electrical capacity per energy source of suspect data
DE_re.groupby(['comment', 'energy_source'])['electrical_capacity_kW'].sum() / 1000

comment     energy_source                
            Bioenergy and renewable waste     7234.835684
            Geothermal                          38.935000
            Hydro                             1564.354359
            Other fossil fuels                 246.818000
            Solar                            39410.769831
            Wind                             46634.729013
            solar                              279.792455
R_1,        Bioenergy and renewable waste       62.684100
            Geothermal                           3.800000
            Hydro                                6.281050
            Other fossil fuels                   0.006000
            Solar                              477.533366
            Wind                              8860.719460
            solar                             1597.011551
R_1, R_3,   Bioenergy and renewable waste     1020.468990
            Hydro                               83.317700
            Other fossil fuels

In [11]:
# Summarize electrical capacity per energy source of cleaned data (MW)
DE_re_clean.groupby(['energy_source'])['electrical_capacity_kW'].sum() / 1000

energy_source
Bioenergy and renewable waste     7234.835684
Geothermal                          38.935000
Hydro                             1564.354359
Other fossil fuels                 246.818000
Solar                            39410.769831
Wind                             46634.729013
solar                              279.792455
Name: electrical_capacity_kW, dtype: float64

# 6. Capacity time series

This section creates a daily and yearly time series of the cumulated installed capacity by energy source. This data will be part of the output and will be compared in a plot for validation in the next section.

In [12]:
# Additional column for chosing energy sources for timeseries
DE_re_clean['temp_energy_source'] = DE_re_clean['energy_source']

# Time series for on- and offshore wind should be separated
idx_wind = DE_re_clean[DE_re_clean.energy_source == 'Wind'].index
DE_re_clean.loc[idx_wind, 'temp_energy_source'] = DE_re_clean.loc[
                                                  idx_wind, 'energy_source_subtype']

# Set energy source for which time series should be generated
energy_sources = ['Bioenergy and renewable waste', 'Onshore', 'Offshore', 'Solar',
                  'Geothermal', 'Hydro']

In [13]:
# Set date range for which the time series should be generated
range_yearly = pd.date_range(start='1990-01-01', end='2016-01-01', freq='A')
range_daily = pd.date_range(start='2005-01-01', end='2016-01-31', freq='D')

# Set range of time series as index
timeseries_yearly = pd.DataFrame(index=range_yearly)
timeseries_daily = pd.DataFrame(index=range_daily)

In [14]:
# Create cumulated time series per energy source for both yearly and daily time series
for gtype in energy_sources:
    temp = (DE_re_clean[['commissioning_date', 'electrical_capacity_kW']]
            .loc[DE_re_clean['temp_energy_source'].isin([gtype])])
    
    temp_timeseries = temp.set_index('commissioning_date')
    
    # Create cumulated time series per energy_source and year
    timeseries_yearly['{0}_DE_capacity'.format(gtype)] = temp_timeseries.resample(
        'A', how='sum').cumsum().fillna(method='ffill') / 1000

    # Create cumulated time series per energy_source and day
    timeseries_daily['{0}_DE_capacity'.format(gtype)] = temp_timeseries.resample(
        'D', how='sum').cumsum().fillna(method='ffill') / 1000

/home/frauke/.local/lib/python3.5/site-packages/IPython/kernel/__main__.py:10: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
/home/frauke/.local/lib/python3.5/site-packages/IPython/kernel/__main__.py:14: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()


In [15]:
# Set index name
timeseries_yearly.columns.rename('timestamp', inplace=True)
timeseries_daily.columns.rename('timestamp', inplace=True)

# Shorten timestamp to year for the yearly timeseries    
timeseries_yearly.index = pd.to_datetime(timeseries_yearly.index, format="%Y").year

# 7. Plots

## 7.1 Cumulated capacity validation

Compare cumulated capacity per country and energy source of the power plant list to data from the International Renewable Energy Agency (IRENA) and data from ENTSO-E and Eurostat, already compiled in another OPSD datapackage.

### 7.1.1 Download and read data for validaion

#### IRENA
Automatic Download of IRENA data does not work yet, so download the file Europe.csv manually with this link and put the file into the folder _input_:
http://public.tableau.com/profile/irena.resource#!/vizhome/ExportIRENAData/Europe.csv

In [16]:
irena_df = pd.read_csv('input/Europe.csv',
                       sep = ",",
                       decimal = ".",
                       thousands = ",",
                       encoding = 'UTF8',
                       header = 0)

In [17]:
# Select just the required values for capacity
idx_irena = irena_df[(irena_df['Flow'] == 'Total capacity (MWe)')].index
irena_df = irena_df.loc[idx_irena,('Country / Area','Sub Technology','Year','Value')]

In [18]:
# Read column and value translation list
columnnames = pd.read_csv('input/column_translation_list.csv')
valuenames = pd.read_csv('input/value_translation_list.csv')

In [19]:
# Create dictionary for column translation
idx_Europe = columnnames[columnnames['country'] == 'Europe'].index
column_dict = columnnames.loc[idx_Europe].set_index('original_name')[
                                                           'opsd_name'].to_dict()

# Create dictionnary in order to adapt energy_source_subtype names
idx_Europe = valuenames[valuenames['data_source'] == 'IRENA'].index
value_dict = valuenames.loc[idx_Europe].set_index('original_name')[
                                                  'opsd_name'].to_dict()

# Create dictionnary in order to assign energy_source to its subtype
energy_source_dict = valuenames.loc[idx_Europe].set_index('opsd_name')[
                                                          'energy_source'].to_dict()
energy_source_dict

{'Biomass and biogas': 'Bioenergy and renewable waste',
 'Concentrated solar power': 'Solar',
 'DE': nan,
 'DK': nan,
 'FR': nan,
 'Geothermal': 'Geothermal',
 'Hydro': 'Hydro',
 'Marine': 'Marine',
 'Offshore': 'Wind',
 'Onshore': 'Wind',
 'PL': nan,
 'Photovoltaics': 'Solar'}

In [20]:
# Translate columns by list 
irena_df.rename(columns = column_dict, inplace = True)

# Replace energy_source names
irena_df.replace(value_dict, inplace=True)

# Create new column for energy_source
irena_df['energy_source'] = irena_df['energy_source_subtype']

# Fill this with the energy source instead of subtype information
irena_df.energy_source.replace(energy_source_dict, inplace=True)

In [21]:
# Group and summarize data frame by generation type and installed capacity
irena_sum_df = irena_df.groupby(['country','energy_source','year'],
                                as_index = False)['electrical_capacity_MW'].sum()

In [22]:
# Add column to keep track of the source
irena_sum_df['data_source'] = 'IRENA'

#### Aggregated capacity from OPSD datapackage
Download csv-file from OPSD-page which includes ENTSO-E and Eurostat data of cumulated capacity per source of European countries and put it into the folder _input_: <br>
http://data.open-power-system-data.org/national_generation_capacities/ <br>
_This will be done automatically as soon as the passwords are not utilized anymore_

In [23]:
opsd_df = pd.read_csv('input/aggregated_capacity.csv',
                      sep = ",",
                      decimal = ".",
                      encoding = 'UTF8',
                      header = 0)

In [24]:
# Translate columns by list 
opsd_df.rename(columns = column_dict, inplace = True)

# Choose just the required translation terms for this data source
idx_opsd = valuenames[valuenames['data_source'] == 'OPSD'].index

# Create dictionnary in order to change install_type names 
value_dict = valuenames.loc[idx_opsd].set_index('original_name')['opsd_name'].to_dict()

# Replace energy_source names
opsd_df.replace(value_dict, inplace = True)

In [25]:
# Group and summarize data frame by generation type and installed capacity
e_source = pd.Series(['Biomass and biogas','Wind','Solar','Geothermal','Hydro','Marine'])
recent_years = (2014,2015)

In [26]:
idx_opsd = opsd_df[(opsd_df['year'].isin(recent_years))
                    & (opsd_df['energy_source'].isin(e_source))].index

In [27]:
# only required columns are chosen, and only rows of recent years
opsd_df = opsd_df.loc[idx_opsd].drop(['Unnamed: 0','source_type',
                      'type','technology_level_0','technology_level_1',
                      'technology_level_2','technology_level_3'], axis = 1)

# index is not required any more
opsd_df = opsd_df.reset_index(drop = True)

### 7.1.2 Prepare cumulated capacity for comparison
Installed capacity is summed per country and energy source to compare it to the other sources for cumulated capacity. This gives an impression how complete the country power plant list datasets are.

Germany DE

In [28]:
# Summarize per energy_source until end of 2015 without marked entries
DE_re_sum = DE_re_clean[(DE_re_clean['commissioning_date'] <= '2015-12-31')].groupby([
        'energy_source'], as_index = False)['electrical_capacity_kW'].sum()

# Transform kW to MW
DE_re_sum['electrical_capacity_kW'] /= 1000

# Adapt column name
DE_re_sum.rename(columns={'electrical_capacity_kW': 'electrical_capacity_MW'},
                inplace=True)

# Add information until which year (including) the capacity is summarized
DE_re_sum['year'] = 2015
DE_re_sum['country'] = 'DE'

DE_re_sum['data_source'] = 'German TSOs'

DE_re_sum

,energy_source,electrical_capacity_MW,year,country,data_source
0,Bioenergy and renewable waste,7223.201684,2015,DE,German TSOs
1,Geothermal,33.435000,2015,DE,German TSOs
2,Hydro,1563.121819,2015,DE,German TSOs
3,Other fossil fuels,244.102000,2015,DE,German TSOs
4,Solar,39337.659026,2015,DE,German TSOs
5,Wind,44547.715513,2015,DE,German TSOs


Denmark DK

In [29]:
# Summarize per energy_source until end of 2015
DK_re_sum = DK_re[(DK_re['commissioning_date'] <= '2015-12-31')].groupby([
                   'energy_source','data_source'], as_index = False)[
                  'electrical_capacity_kW'].sum()

# Transform kW to MW
DK_re_sum['electrical_capacity_kW'] /= 1000

# Adapt column name
DK_re_sum.rename(columns={'electrical_capacity_kW': 'electrical_capacity_MW'},
                inplace=True)

# Add information until which year (including) the capacity is summarized
DK_re_sum['year'] = 2015
DK_re_sum['country'] = 'DK'

DK_re_sum

,energy_source,data_source,electrical_capacity_MW,year,country
0,solar,Energinet.dk,NaN,2015,DK
1,wind,Energistyrelsen,5059.535,2015,DK


France FR

In [30]:
# Summarize per energy_source. Data until 31.12.2014 is provided by the data source
FR_re_sum = FR_re.groupby(['energy_source','data_source'], as_index = False)[
                           'electrical_capacity_kW'].sum()

# Transform kW to MW
FR_re_sum['electrical_capacity_kW'] /= 1000

# Adapt column name
FR_re_sum.rename(columns={'electrical_capacity_kW': 'electrical_capacity_MW'},
                inplace=True)

# Add information until which year (including) the capacity is summarized
FR_re_sum['year'] = 2014
FR_re_sum['country'] = 'FR'

FR_re_sum

,energy_source,data_source,electrical_capacity_MW,year,country
0,Biomass and biogas,gouv.fr,1343.799800,2014,FR
1,Geothermal,gouv.fr,9.000000,2014,FR
2,Hydro,gouv.fr,2074.297880,2014,FR
3,Photovoltaics,gouv.fr,8602.080614,2014,FR
4,Wind,gouv.fr,10573.327000,2014,FR


Poland PL

In [31]:
# Summarize per energy_source. Data until 31.12.2015 is provided by the data source
PL_re_sum = PL_re.groupby(['energy_source','data_source'], as_index = False)[
                           'electrical_capacity_kW'].sum()

# Transform kW to MW
PL_re_sum['electrical_capacity_kW'] /= 1000

# Adapt column name
PL_re_sum.rename(columns={'electrical_capacity_kW': 'electrical_capacity_MW'},
                inplace=True)

# Add information until which year (including) the capacity is summarized
PL_re_sum['year'] = 2015
PL_re_sum['country'] = 'PL'

PL_re_sum

,energy_source,data_source,electrical_capacity_MW,year,country
0,Bioenergy and renewable waste,Urzad Regulacji Energetyki,802.459,2015,PL
1,Hydro,Urzad Regulacji Energetyki,328.498,2015,PL
2,Solar,Urzad Regulacji Energetyki,46.592,2015,PL
3,Wind,Urzad Regulacji Energetyki,3361.635,2015,PL


In [32]:
# Merge dataframes
capacities = pd.concat([irena_sum_df, opsd_df,
                        DE_re_sum, DK_re_sum, FR_re_sum, PL_re_sum])

# Get rid von all indices
capacities = capacities.reset_index(drop = True)

### 7.1.3 Plot cumulated capacity
One comparison plot for each country: Cumulated capacity in MW per energy source from different sources.

In [33]:
plot_country = 'DE'

In [34]:
# chose capacities of respective country and recent years
idx_country = capacities[(capacities['country'] == plot_country) 
                        & (capacities['year'].isin(recent_years))].index

# prepare data frame for plotting
cap_country = capacities.loc[idx_country].drop(['country',
                                                'capacity_definition'],
                                               axis = 1)

cap_country = cap_country.reset_index(drop = True)

cap_country

,data_source,electrical_capacity_MW,energy_source,year
0,IRENA,8977.000000,Bioenergy and renewable waste,2014
1,IRENA,9132.000000,Bioenergy and renewable waste,2015
2,IRENA,24.000000,Geothermal,2014
3,IRENA,37.700000,Geothermal,2015
4,IRENA,11234.000000,Hydro,2014
5,IRENA,11234.000000,Hydro,2015
6,IRENA,38235.500000,Solar,2014
7,IRENA,39635.500000,Solar,2015
8,IRENA,39193.000000,Wind,2014
9,IRENA,44947.000000,Wind,2015


In [35]:
# plot
p = Bar(cap_country, label='energy_source', values='electrical_capacity_MW',
        group=['data_source','year'], legend='top_left', 
        title=plot_country)

show(p)

In [36]:
plot_country = 'DK'

In [37]:
# chose capacities of respective country and recent years
idx_country = capacities[(capacities['country'] == plot_country) 
                        & (capacities['year'].isin(recent_years))].index

# prepare data frame for plotting
cap_country = capacities.loc[idx_country].drop(['country',
                                                'capacity_definition'],
                                               axis = 1)

cap_country = cap_country.reset_index(drop = True)

cap_country

,data_source,electrical_capacity_MW,energy_source,year
0,IRENA,1267.000,Bioenergy and renewable waste,2014
1,IRENA,1267.000,Bioenergy and renewable waste,2015
2,IRENA,9.000,Hydro,2014
3,IRENA,9.000,Hydro,2015
4,IRENA,607.000,Solar,2014
5,IRENA,790.000,Solar,2015
6,IRENA,4887.000,Wind,2014
7,IRENA,5063.000,Wind,2015
8,entsoe,10.000,Hydro,2014
9,entsoe,10.000,Hydro,2014


In [38]:
# plot
p = Bar(cap_country, label='energy_source', values='electrical_capacity_MW',
        group=['data_source','year'], legend='top_left', 
        title=plot_country)

show(p)

In [39]:
plot_country = 'FR'

In [40]:
# chose capacities of respective country and recent years
idx_country = capacities[(capacities['country'] == plot_country) 
                        & (capacities['year'].isin(recent_years))].index

# prepare data frame for plotting
cap_country = capacities.loc[idx_country].drop(['country',
                                                'capacity_definition'],
                                               axis = 1)

cap_country = cap_country.reset_index(drop = True)

cap_country

,data_source,electrical_capacity_MW,energy_source,year
0,IRENA,1115.000000,Bioenergy and renewable waste,2014
1,IRENA,1267.000000,Bioenergy and renewable waste,2015
2,IRENA,25294.000000,Hydro,2014
3,IRENA,25421.000000,Hydro,2015
4,IRENA,240.000000,Marine,2014
5,IRENA,241.000000,Marine,2015
6,IRENA,5654.300000,Solar,2014
7,IRENA,6549.300000,Solar,2015
8,IRENA,9068.000000,Wind,2014
9,IRENA,10358.000000,Wind,2015


In [41]:
# plot
p = Bar(cap_country, label='energy_source', values='electrical_capacity_MW',
        group=['data_source','year'], legend='top_left', 
        title=plot_country)

show(p)

In [42]:
plot_country = 'PL'

In [43]:
# chose capacities of respective country and recent years
idx_country = capacities[(capacities['country'] == plot_country) 
                        & (capacities['year'].isin(recent_years))].index

# prepare data fraem for plotting
cap_country = capacities.loc[idx_country].drop(['country',
                                                'capacity_definition'],
                                               axis = 1)

cap_country = cap_country.reset_index(drop = True)

In [44]:
# plot
p = Bar(cap_country, label='energy_source', values='electrical_capacity_MW',
        group=['data_source','year'], legend='top_left', 
        title=plot_country)

show(p)

## 7.2 Capacity time series DE - validation
The yearly capacity time series by energy source derived from the power plant list is validated plotting it in comparison to a timeseries published by the German Federal Ministry for Economic Affairs and Energy (BMWi, Bundesministeriums für Wirtschaft und Energie) [Time series for the development of renewable energies in Germany](http://www.erneuerbare-energien.de/EE/Navigation/DE/Service/Erneuerbare_Energien_in_Zahlen/Zeitreihen/zeitreihen.html)

**Download BMWi time series** *(If an error message is shown, check if the url_bmwi_stat is still correct.)*

In [45]:
# Defining URL
url_bmwi_stat = 'http://www.erneuerbare-energien.de/EE/Redaktion/DE/'\
                'Downloads/zeitreihen-zur-entwicklung-der-erneuerbaren-'\
                'energien-in-deutschland-1990-2015-excel.xlsx;jsessionid='\
                'FFE958ADA709DCBFDD437C8A8FF7D90B?__blob=publicationFile&v=6'

# Reading BMWi data
bmwi_stat = pd.read_excel(url_bmwi_stat,
                          sheetname='4', 
                          header=7,
                          skip_footer=8,
                          index_col=0)

# Transpose data frame and set column names
bmwi_df = bmwi_stat.T
bmwi_df.columns = ['bmwi_hydro', 'bmwi_wind_onshore', 'bmwi_wind_offshore',
                   'bmwi_solar', 'bmwi_biomass', 'bmwi_biomass_liquid',
                   'bmwi_biomass_gas', 'bmwi_sewage_gas', 'bmwi_landfill_gas',
                   'bmwi_geothermal', 'bmwi_total']

# Set year as index
bmwi_df.index = pd.to_datetime(bmwi_df.index, format="%Y").year

** Calculate the absolute deviation between BMWi and OPSD timeseries**

In [46]:
valuation = pd.concat([bmwi_df, timeseries_yearly], axis=1)
valuation = valuation.fillna(0)

In [47]:
valuation.head(2)

,bmwi_hydro,bmwi_wind_onshore,bmwi_wind_offshore,bmwi_solar,bmwi_biomass,bmwi_biomass_liquid,bmwi_biomass_gas,bmwi_sewage_gas,bmwi_landfill_gas,bmwi_geothermal,bmwi_total,Bioenergy and renewable waste_DE_capacity,Onshore_DE_capacity,Offshore_DE_capacity,Solar_DE_capacity,Geothermal_DE_capacity,Hydro_DE_capacity
1990,3982,55,0,2,64,0,1,5,59,0,4168,68.989,11.461,0.0,0.529209,0.0,428.58394
1991,4033,106,0,2,64,0,2,5,64,0,4276,75.205,21.646,0.0,1.357654,0.0,448.47634


In [48]:
# Calculate absolute deviation for each year and energy source

valuation['absolute_wind_onshore'] = (valuation['Onshore_DE_capacity']
                                      - valuation['bmwi_wind_onshore']).fillna(0)

valuation['absolute_wind_offshore'] = (valuation['Offshore_DE_capacity']
                                       - valuation['bmwi_wind_offshore']).fillna(0)

valuation['absolute_solar'] = (valuation['Solar_DE_capacity']
                               - valuation['bmwi_solar']).fillna(0)

valuation['absolute_hydro'] = (valuation['Hydro_DE_capacity']
                               - valuation['bmwi_hydro']).fillna(0)

valuation['absolute_geothermal'] = (valuation['Geothermal_DE_capacity']
                                    - valuation['bmwi_geothermal']).fillna(0)

valuation['absolute_biomass'] = (valuation['Bioenergy and renewable waste_DE_capacity']
                                 - (valuation['bmwi_biomass']
                                    + valuation['bmwi_biomass_liquid']
                                    + valuation['bmwi_biomass_gas']
                                    + valuation['bmwi_sewage_gas']
                                    + valuation['bmwi_landfill_gas'])).fillna(0)

valuation['absolute_total'] = ((valuation['Bioenergy and renewable waste_DE_capacity']
                                + valuation['Onshore_DE_capacity']
                                + valuation['Offshore_DE_capacity']
                                + valuation['Solar_DE_capacity']
                                + valuation['Geothermal_DE_capacity']
                                + valuation['Hydro_DE_capacity']
                                ) - (valuation['bmwi_total'])).fillna(0)

**Plotting the absolute deviation**

The graph shows the cumulative absolute deviation of the electrical capacity in MW between the OPSD data set of renewable power plants and the BMWI statistic from 1990 until 2015.<br>
*A positive deviation means the OPSD data set has more capacity than the BMWI statistics.*

In [49]:
#Plot settings for absolute deviation
deviation_columns = ['absolute_wind_onshore','absolute_wind_offshore',
                     'absolute_solar','absolute_hydro','absolute_biomass',
                     'absolute_geothermal','absolute_total']

dataplot = valuation[deviation_columns]

deviation = Line(dataplot,
                 y=deviation_columns,
                 dash=deviation_columns,
                 color=deviation_columns,
                 title="Deviation between data set and BMWI statistic (negative => BMWI value higher)",
                 ylabel='Deviation in MW',
                 xlabel='From 1990 till 2015',
                 legend=True)

In [50]:
# Show Plot for absolute deviation
show(deviation)

** Calculate the relative deviation between both timeseries**

In [51]:
# Relative deviation
valuation['relative_wind_onshore'] = (valuation['absolute_wind_onshore'] / valuation['bmwi_wind_onshore']).fillna(0)

valuation['relative_wind_offshore'] = (valuation['absolute_wind_offshore'] / valuation['bmwi_wind_offshore']).fillna(0)

valuation['relative_solar'] = (valuation['absolute_solar'] / (valuation['bmwi_solar'])).fillna(0)

valuation['relative_hydro'] = (valuation['absolute_hydro'] / (valuation['bmwi_hydro'])).fillna(0)

valuation['relative_geothermal'] = (valuation['absolute_geothermal'] / (valuation['bmwi_geothermal'])).fillna(0)

valuation['relative_biomass'] = (valuation['absolute_biomass'] / (valuation['bmwi_biomass'])).fillna(0)

valuation['relative_total'] = (valuation['absolute_total'] / (valuation['bmwi_total'])).fillna(0)

In [52]:
# Plot settings relative deviation
relative_column = ['relative_wind_onshore','relative_wind_offshore',
                   'relative_solar','relative_hydro','relative_biomass',
                   'relative_total']

dataplot2 = valuation[relative_column]

relative = Line(dataplot2 * 100,
                y=relative_column,
                dash=relative_column,
                color=relative_column,
                title="Deviation between data set and BMWI statistic (negative => BMWI value higher)",
                ylabel='Relative difference in percent',
                xlabel='From 1990 till 2015',
                legend=True)

**Plotting the relative deviation**

The graph shows the relative difference of the electrical capacity between the OPSD data set of renewable power plants and the BMWI statistic from 1990 until 2015.<br>
*A positive value means the OPSD data set has more capacity than the BMWI statistics.*

In [8]:
# Show Plot for relative deviation
show(relative)

NameError: name 'relative' is not defined

**Saving deviation results in XLSX-file**

In [54]:
# write results as Excel file
valuation.to_excel('validation_report.xlsx', sheet_name='Capacities_1990_2015')

# 8. Harmonize format for output files

## 8.1 Columns
A similar order of columns is pursued for all country lists.

In [11]:
DE_re = DE_re.loc[:,('commissioning_date','decommissioning_date',
                     'energy_source','energy_source_subtype',
                     'electrical_capacity_kW','thermal_capacity_kW',
                     'voltage_level','tso','dso','dso_id', 'eeg_id','bnetza_id',
                     'federal_state','postcode','municipality_code','municipality',
                     'address','address_number',
                     'utm_zone','utm_east','utm_north','latitude','longitude',
                     'data_source','comment')]

In [12]:
DK_re = DK_re.loc[:,('commissioning_date',
                     'energy_source','energy_source_subtype',
                     'electrical_capacity_kW',
                     'tso','dso','gsrn_id',
                     'postcode','municipality_code','municipality',
                     'address','address_number',
                     'utm_east','utm_north','latitude','longitude',
                     'hub_height','rotor_diameter','manufacturer','model',
                     'data_source')]

In [13]:
FR_re = FR_re.loc[:,('municipality_code','municipality',
                     'energy_source','energy_source_subtype',
                     'electrical_capacity_kW','number_of_installations',
                     'latitude','longitude',
                     'data_source')]

In [14]:
PL_re = PL_re.loc[:,('district',
                     'energy_source','energy_source_subtype',
                     'electrical_capacity_kW','number_of_installations',
                     'latitude','longitude',
                     'data_source')]

## 8.2 Accuracy of numbers

**One digit behind the comma for electrical_capacity_kW.**

In [15]:
DE_re['electrical_capacity_kW'] = DE_re['electrical_capacity_kW'
                                       ].map(lambda x: '%.1f' % x)
DE_re['thermal_capacity_kW'] = DE_re['thermal_capacity_kW'
                                    ].map(lambda x: '%.1f' % x)

DK_re['electrical_capacity_kW'] = DK_re['electrical_capacity_kW'
                                       ].map(lambda x: '%.1f' % x)

FR_re['electrical_capacity_kW'] = FR_re['electrical_capacity_kW'
                                       ].map(lambda x: '%.1f' % x)

PL_re['electrical_capacity_kW'] = PL_re['electrical_capacity_kW'
                                       ].map(lambda x: '%.1f' % x)

**Five digits behind the comma for coordinates.**

In [16]:
DE_re['latitude'] = pd.to_numeric(DE_re['latitude'], errors='coerce')
DE_re['longitude'] = pd.to_numeric(DE_re['longitude'], errors='coerce')
DE_re['latitude'] = DE_re['latitude'].map(lambda x: '%.5f' % x)
DE_re['longitude'] = DE_re['longitude'].map(lambda x: '%.5f' % x)
DE_re['utm_east'] = DE_re['utm_east'].map(lambda x: '%.5f' % x)
DE_re['utm_north'] = DE_re['utm_north'].map(lambda x: '%.5f' % x)

# Zone value has to be an integer, thus no digits after the comma
DE_re['utm_zone'] = pd.to_numeric(DE_re['utm_zone'], errors='coerce')
DE_re['utm_zone'] = DE_re['utm_zone'].map(lambda x: '%.0f' % x)

DK_re['latitude'] = pd.to_numeric(DK_re['latitude'], errors='coerce')
DK_re['longitude'] = pd.to_numeric(DK_re['longitude'], errors='coerce')
DK_re['latitude'] = DK_re['latitude'].map(lambda x: '%.5f' % x)
DK_re['longitude'] = DK_re['longitude'].map(lambda x: '%.5f' % x)
DK_re['utm_east'] = DK_re['utm_east'].map(lambda x: '%.5f' % x)
DK_re['utm_north'] = DK_re['utm_north'].map(lambda x: '%.5f' % x)

FR_re['latitude'] = pd.to_numeric(FR_re['latitude'], errors='coerce')
FR_re['longitude'] = pd.to_numeric(FR_re['longitude'], errors='coerce')
FR_re['longitude'] = FR_re['longitude'].map(lambda x: '%.5f' % x)
FR_re['latitude'] = FR_re['latitude'].map(lambda x: '%.5f' % x)

**Dates in the output should be without a timestamp.**

In [17]:
DE_re['commissioning_date'] = DE_re['commissioning_date'].apply(lambda x: x.date())

DK_re['commissioning_date'] = DK_re['commissioning_date'].apply(lambda x: x.date())

In [64]:
#DE_re_clean['decommissioning_date'] = DE_re_clean[pd.to_datetime('decommissioning_date'
 #                                                )].apply(lambda x: x.date())

ValueError: Unknown string format

## 8.3 Sort

In [18]:
# Sort German data frame by commissioning date
DE_re = DE_re.ix[DE_re_clean.commissioning_date.sort_values().index]

# Sort Danish data frame by commissioning date
DK_re = DK_re.ix[DK_re.commissioning_date.sort_values().index]

# Sort French data frame by municipality_code
FR_re = FR_re.ix[FR_re.municipality_code.sort_values().index]

# Sort Polish data frame by district
PL_re = PL_re.ix[PL_re.district.sort_values().index]

## 8.4 Merge country data for SQL-output

In [19]:
# Merge country DataFrames for SQLlite-file
EU_re = pd.concat([DE_re, DK_re, FR_re, PL_re])

# 9. Output
This section finally writes the data package:
* CSV + XLSX + SQLite of the power plant lists
* CSV of the time series
* Meta data (JSON)

## 9.1 Settings

In [20]:
path_package = 'output/renewable_power_plants'

os.makedirs(path_package, exist_ok=True)

## 9.2 Write power plant lists

** Write CSV-files**

_This process will take some time: ~10-15 minutes_

One csv-file for each country. 

In [10]:
# Germany
DE_re.to_csv(path_package + '/renewable_power_plants_DE.csv',
             sep=',',
             decimal='.',
             date_format='%Y-%m-%d',
             encoding='utf-8',
             index=False,
             if_exists="replace")

NameError: name 'path_package' is not defined

In [102]:
# Denmark
DK_re.to_csv(path_package + '/renewable_power_plants_DK.csv',
             sep=',',
             decimal='.',
             date_format='%Y-%m-%d',
             encoding='utf-8',
             index=False,
             if_exists="replace")

In [103]:
# France
FR_re.to_csv(path_package + '/renewable_power_plants_FR.csv',
             sep=',',
             decimal='.',
             date_format='%Y-%m-%d',
             encoding='utf-8',
             index=False,
             if_exists="replace")

In [104]:
# Denmark
PL_re.to_csv(path_package + '/renewable_power_plants_PL.csv',
             sep=',',
             decimal='.',
             date_format='%Y-%m-%d',
             encoding='utf-8',
             index=False,
             if_exists="replace")

** Write XLSX-file**

_This process will take some time: 20 min and more_

All country power plant list will be written in one xlsx-file. Each country power plant list is written in a separate sheet. As the German power plant list has to many entries for one sheet, it will be split in two. An additional sheet includes the explanations of the marker

In [19]:
# Read csv of Marker Explanations
validation = pd.read_csv('input/validation_marker.csv',
                         sep=',', header=0)

In [22]:
DE_re.to_pickle('DE_re_before_output.pickle')
DK_re.to_pickle('DK_re_before_output.pickle')
FR_re.to_pickle('FR_re_before_output.pickle')
PL_re.to_pickle('PL_re_before_output.pickle')
EU_re.to_pickle('EU_re_before_output.pickle')

AttributeError: 'DataFrame' object has no attribute 'to_pickls'

In [23]:
print('Preparing writer')
# Write the results as xlsx file
%time writer = pd.ExcelWriter(path_package + '/renewable_power_plants.xlsx',engine='xlsxwriter',date_format='yyyy-mm-dd')

print('Writing DK')
%time DK_re.to_excel(writer, index=False, sheet_name='DK')

print('Writing FR')
%time FR_re.to_excel(writer, index=False, sheet_name='FR')

print('Writing PL')
%time PL_re.to_excel(writer, index=False, sheet_name='PL')

print('Writing DE part 1')
%time DE_re[:1000000].to_excel(writer, index=False, sheet_name='DE part-1')

#print('Writing DE part 2')
#%time DE_re[1000000:].to_excel(writer, index=False, sheet_name='DE part-2')

print('Writing validation')
# The explanation of validation markers is added as a sheet
%time validation.to_excel(writer, index=False, sheet_name='validation_marker')

print('Saving...')
# Close the Pandas Excel writer and output the Excel file.
%time writer.save()
print('...done!')

Preparing writer
CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 23.3 ms
Writing DK
CPU times: user 1min 51s, sys: 1.21 s, total: 1min 52s
Wall time: 1min 57s
Writing FR
CPU times: user 21.9 s, sys: 12 ms, total: 21.9 s
Wall time: 21.9 s
Writing PL
CPU times: user 292 ms, sys: 0 ns, total: 292 ms
Wall time: 292 ms
Writing DE part 1
CPU times: user 24min 50s, sys: 9.46 s, total: 24min 59s
Wall time: 25min 30s
Writing validation
CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 31.6 ms
Saving...
CPU times: user 11min 8s, sys: 10.3 s, total: 11min 18s
Wall time: 11min 41s
...done!


**Write SQLite**

In [28]:
# Write the results to sqlite database
EU_re.to_sql('renewable_power_plants_germany',
                        sqlite3.connect(path_package +
                                        '/renewable_power_plants.sqlite'),
                        if_exists="replace")

## 9.3 Write time series

In [70]:
# Write daily cumulated time series as csv
timeseries_daily.to_csv(path_package + '/renewable_capacity_timeseries_DE.csv',
                        sep=',',
                        float_format='%.3f',
                        decimal='.',
                        date_format='%Y-%m-%dT%H:%M:%S%z',
                        encoding='utf-8',
                        if_exists="replace")

## 9.4 Write meta data

The data packages meta data are created in the specific JSON format as proposed by the Open Knowledge Foundation. Please see the Frictionless Data project by OKFN (http://data.okfn.org/) and the Data Package specifications (http://dataprotocols.org/data-packages/) for more details.

In order to keep the notebook more readable the metadata is written in the human-readable YAML format using a multi-line string and then parse the string into a Python dictionary and save is as a JSON file.

In [73]:
metadata = """
name: opsd-renewable-power-plants
title: List of renewable power plants
description: >-
    This data package contains a list of renewable energy power plants in lists of 
    renewable energy-based power plants of Germany, Denmark, France and Poland. 
    Germany: More than 1.7 million renewable power plant entries, eligible under the 
    renewable support scheme (EEG). 
    Denmark: Wind and phovoltaic power plants with a high level of detail. 
    France: Aggregated capacity and number of installations per energy source per 
    municipality (Commune). 
    Poland: Summed capacity and number of installations per energy source 
    per municipality (Powiat). 
    Due to different data availability, the power plant lists are of different 
    accurancy and partly provide different power plant parameter. Due to that, the 
    lists are provided as seperate csv-files per country and as separate sheets in the
    excel file. Additionally, the datapackage includes a daily time series of cumulated
    installed capacity per energy source type for Germany. All data processing is 
    conducted in python and pandas and has been documented in the Jupyter notebooks 
    linked below. 
version: "2016-09-09"
keywords: [master data register,power plants,renewables,germany,denmark,france,poland]
geographical-scope: Germany,Denmark,France,Poland
resources:
    - path: renewable_power_plants_DE.csv
      format: csv
      schema:         
          fields:
            - name: commissioning_date
              type: date
              format: YYYY-MM-DD
            - name: decommissioning_date
              type: date
              format: YYYY-MM-DD
            - name: energy_source
              description: Type of energy source (e.g. Wind)
              type: string
            - name: energy_source_subtype
              description: Subtype of energy source (e.g. Onshore)
              type: string
            - name: electrical_capacity_kW
              description: Installed electrical capacity in kW
              type: number
              format: float
              unit: kW
            - name: thermal_capacity
              description: Installed thermal capacity in kW
              type: number
              format: float
              unit: kW
            - name: voltage_level
              description: Voltage level of grid connection
              type: string
            - name: tso
              description: Name of transmission system operator of the area the plant is located
              type: string
            - name: dso
              description: Name of distribution system operator of the region the plant is located in
              type: string
            - name: dso_id
              description: Company number of German distribution grid operator
              type: string
            - name: eeg_id
              description: Power plant EEG (German feed-in tariff law) remuneration number
              type: string
            - name: bnetza_id
              description: Power plant identification number by BNetzA
              type: string
            - name: federal_state
              description: Name of German administrative level 'Bundesland'
              type: string
            - name: postcode
              description: German zip-code
              type: string
            - name: municipality_code
              description: German Gemeindenummer (municipalitiy number)
              type: string
            - name: municipality
              description: Name of German Gemeinde (municipality)
              type: string
            - name: address
              description: Street name or name of land parcel
              type: string
            - name: address_number
              description: House number or number of land parcel
              type: string
            - name: utm_zone
              description: Universal Transverse Mercator zone value
              type: 
            - name: utm_east
              description: Coordinate in Universal Transverse Mercator (east)
              type: numeric
              format: float
            - name: utm_north
              description: Coordinate in Universal Transverse Mercator (north)
              type: numeric
              format: float
            - name: latitude
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: longitude
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: data_source
              description: Source of database entry
              type: string
            - name: comment
              description: Validation comments
              type: string
    - path: renewable_power_plants_DK.csv
      format: csv
      schema:
          fields:
            - name: commissioning_date
              type: date
              format: YYYY-MM-DD
            - name: energy_source
              description: Type of energy source (e.g. Wind)
              type: string
            - name: energy_source_subtype
              description: Subtype of energy source (e.g. Onshore)
              type: string
            - name: electrical_capacity_kW
              description: Installed electrical capacity in kW
              type: number
              format: float
            - name: tso
              description: Name of transmission system operator of the area the plant is located
              type: string
            - name: dso
              description: Name of distribution system operator of the region the plant is located in
              type: string
            - name: gsrn_id
              description: Danish wind turbine identifier number (GSRN)
              type: number
              format: integer
            - name: postcode
              description: Danish zip-code
              type: string
            - name: municipality_code
              description: Danish 3-digit Kommune-Nr
              type: string
            - name: municipality
              description: Name of Danish Kommune
              type: string
            - name: address
              description: Street name or name of land parcel
              type: string
            - name: address_number
              description: House number or number of land parcel
              type: string
            - name: utm_east
              description: Coordinate in Universal Transverse Mercator (east)
              type: numeric
              format: float
            - name: utm_north
              description: Coordinate in Universal Transverse Mercator (north)
              type: numeric
              format: float
            - name: latitude
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: longitude
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: hub_height
              description: Wind turbine hub heigth in m
              type: numeric
              format: float
            - name: rotor_diameter
              description: Wind turbine rotor diameter in m
              type: numeric
              format: float
            - name: manufacturer
              description: Company that has built the wind turbine
              type: string
            - name: model
              description: Wind turbind model type
              type: string
            - name: data_source
              description: Source of database entry
              type: string
    - path: renewable_power_plants_FR.csv
      format: csv
      schema:
          fields:
            - name: municipality_code
              description: French 5-digit INSEE code for Communes
              type: string
            - name: municipality
              description: Name of French Commune
              type: string
            - name: energy_source
              description: Type of energy source (e.g. Wind)
              type: string
            - name: energy_source_subtype
              description: Subtype of energy source (e.g. Onshore)
              type: string
            - name: electrical_capacity_kW
              description: Installed electrical capacity in kW
              type: number
              format: float
            - name: number_of_installations
              description: Number of installations of the energy source subtype in the municipality
              type: number
              format: integer
            - name: latitude
              description: Latitude coordinates
              type: geopoint
              format: lat
            - name: longitude
              description: Longitude coordinates 
              type: geopoint
              format: lon
            - name: data_source
              description: Source of database entry
              type: string
    - path: renewable_power_plants_PL.csv
      format: csv
      schema:
          fields:
            - name: district
              description: Name of the Polish powiat
              type: string
            - name: energy_source
              description: Type of energy source (e.g. Wind)
              type: string
            - name: energy_source_subtype
              description: Subtype of energy source (e.g. Onshore)
              type: string
            - name: electrical_capacity_kW
              description: Installed electrical capacity in kW
              type: number
              format: float
            - name: number_of_installations
              description: Number of installations of the energy source subtype in the district
              type: number
              format: integer
            - name: data_source
              description: Source of database entry
              type: string
    - path: renewable_power_plants.xlsx
      format: xlsx
    - path: validation_marker.csv
      format: csv
      mediatype: text/csv
      schema:         
          fields:
            - name: Validation_Marker
              description: Name of validation marker utilized in column comment in the renewable_power_plant_germany.csv
              type: string
            - name: Explanation
              description: Comment explaining meaning of validation marker
              type: string
    - path: renewable_capacity_germany_timeseries.csv
      format: csv
      mediatype: text/csv
      schema:         
          fields:
            - name: timestamp
              description: Start time of the day
              type: datetime
              format: YYYY-MM-DDThh:mm:ssZ
            - name: capacity_biomass_de
              description: Cumulated biomass electrical capacity
              type: number
            - name: capacity_wind_onshore_de
              description: Cumulated wind onshore capacity
              type: number
            - name: capacity_wind_offshore_de
              description: Cumulated wind offshore capacity
              type: number
            - name: capacity_solar_de
              description: Cumulated solar capacity
              type: number                
            - name: capacity_gas_de
              description: Cumulated gas electrical capacity
              type: number  
            - name: capacity_geothermal_de
              description: Cumulated geothermal electrical capacity
              type: number 
            - name: capacity_hydro_de
              description: Cumulated hydro capacity
              type: number  
licenses:
    - type: MIT license
      url: http://www.opensource.org/licenses/MIT
sources:
    - name: BNetzA
      web: http://www.bundesnetzagentur.de/cln_1422/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/Anlagenregister/Anlagenregister_Veroeffentlichung/Anlagenregister_Veroeffentlichungen_node.html
      source: Bundesnetzagentur register of renewable power plants (excl. PV)
    - name: BNetzA_PV
      web: http://www.bundesnetzagentur.de/cln_1431/DE/Sachgebiete/ElektrizitaetundGas/Unternehmen_Institutionen/ErneuerbareEnergien/Photovoltaik/DatenMeldgn_EEG-VergSaetze/DatenMeldgn_EEG-VergSaetze_node.html    
      source: Bundesnetzagentur register of PV power plants
    - name: TransnetBW, TenneT, Amprion, 50Hertz, Netztransparenz.de
      web: https://www.netztransparenz.de/de/Anlagenstammdaten.htm
      source: Netztransparenz.de - information platform of German TSOs (register of renewable power plants in their control area)
    - name: Postleitzahlen Deutschland
      web: http://www.suche-postleitzahl.org/downloads
      source: Zip codes of Germany linked to geo-information
    - name: Energinet.dk
      web: http://www.energinet.dk/SiteCollectionDocuments/Danske%20dokumenter/El/SolcelleGraf.xlsx
      source: register of Danish wind power plants
    - name: Energistyrelsen
      web: http://www.ens.dk/sites/ens.dk/files/info/tal-kort/statistik-noegletal/oversigt-energisektoren/stamdataregister-vindmoeller/anlaegprodtilnettet.xls    
      source: ens.dk - register of Danish PV power plants
    - name: GeoNames
      web: http://download.geonames.org/export/zip/DK.zip
      source: geonames.org
    - name: French Ministery of the Environment, Energy and the Sea
      web: http://www.statistiques.developpement-durable.gouv.fr/energie-climat/r/energies-renouvelables.html?tx_ttnews[tt_news]=20647
      source: register of renewable power plants by generation type and municipality
    - name: OpenDataSoft
      web: http://public.opendatasoft.com/explore/dataset/correspondance-code-insee-code-postal/download/'\
           '?format=csv&refine.statut=Commune%20simple&timezone=Europe/Berlin&use_labels_for_header=true
      source: Code Postal - Code INSEE
    - name: Urzad Regulacji Energetyki (URE)
      web: http://www.ure.gov.pl/uremapoze/mapa.html
      source: Energy Regulatory Office of Poland
maintainers:
    - name: Frauke Wiese
      email: frauke.wiese@uni-flensburg.de
      web: http://open-power-system-data.org/
views: True
openpowersystemdata-enable-listing: True
opsd-jupyter-notebook-url: https://github.com/Open-Power-System-Data/datapackage_renewable_power_plants/blob/2016-08-25/main.ipynb
opsd-changes-to-last-version: Included Danish, French and Polish data
"""

metadata = yaml.load(metadata)

datapackage_json = json.dumps(metadata, indent=4, separators=(',', ': '))

# Write the information of the metadata
with open(os.path.join(path_package, 'datapackage.json'), 'w') as f:
    f.write(datapackage_json)